In [19]:
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch import save
import torch
import tqdm

## AI

In [21]:
dataset = []
data = np.loadtxt("connectfour/connectfour.data", dtype=str)
for i in range(len(data)):
    dataset.append(list(data[i].replace('b','0').replace('x','1').replace('loss','0,1,0').replace('o','-1').replace('win','1,0,0').replace("draw",'0,1,0').split(',')))
    
# datasetX = np.array(dataset)[:,:-3][-16:,]
# datasety = np.array(dataset)[:,-3:][-16:,]

datasetX = np.array(dataset)[:,:-3]
datasety = np.array(dataset)[:,-3:]
datasetX.shape

(67557, 42)

In [22]:
train_datasetX = datasetX[:64000]
test_datasetX = datasetX[:-64000]
train_datasety = datasety[:64000]
test_datasety = datasety[:-64000]

In [23]:
class Train_Dataset(Dataset):
    def __init__(self):
        self.dataset = dataset
        self.X = train_datasetX.astype('float32')
        self.y = train_datasety.astype('float32')
        
    def __getitem__(self, index):
        X = torch.from_numpy(self.X).cuda()
        y = torch.from_numpy(self.y).cuda()
        
        return X, y

    def __len__(self):
        return len(self.y)
    
class Test_Dataset(Dataset):
    def __init__(self):
        self.dataset = dataset
        self.X = test_datasetX.astype('float32')
        self.y = test_datasety.astype('float32')
        
    def __getitem__(self, index):
        X = torch.from_numpy(self.X).cuda()
        y = torch.from_numpy(self.y).cuda()
        
        return X, y

    def __len__(self):
        return len(self.y)

In [24]:
traindata = Train_Dataset()
testdata = Test_Dataset()
traindataLoader = DataLoader(traindata,batch_size=64, shuffle =True)
testdataLoader = DataLoader(testdata,batch_size=8, shuffle =True)

In [26]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(42, 500)
        self.relu1 = nn.LeakyReLU()
        self.linear2 = nn.Linear(500, 1000)
        self.relu2 = nn.LeakyReLU()
        self.linear3 = nn.Linear(1000, 3)
        self.drop1   = nn.Dropout(0.15)

    def forward(self, x):
        x = self.linear1(x)
        x = self.relu1(x)
        x = self.linear2(x)
        x = self.relu2(x)
        x = self.drop1(x)
        x = self.linear3(x)
        return x

In [27]:
net = Net().cuda()

In [15]:
torch.cuda.empty_cache()

In [30]:
test_loss = nn.MSELoss().cuda()
loss = nn.SmoothL1Loss().cuda()
trainer = torch.optim.Adam(net.parameters(), lr=0.0001)

num_epochs = 10

train_score_list = []
test_score_list = []

for epoch in tqdm.tqdm(range(num_epochs)):
    loss_epoch = []
    t = 0
    for x, y in traindataLoader:
        net.train()
        t = t + 1
        l = loss(net(x),y)
        loss_epoch.append(l)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    
    print(str(epoch) +" : " + str((sum(loss_epoch)/t).item())[:6])
    train_score =  float(str((sum(loss_epoch)/t).item())[:6])
    train_score_list.append(train_score)   
    
    
    test_score = 0
    for x, y in testdataLoader:
        net.eval()
        with torch.no_grad(): 
            output = net(x) 
        MSE = test_loss(output ,y)
#         print(MSE)

        test_score += MSE.item()
    test_score_list.append(test_score/3557) 
    print(test_score/3557)
    name = str(epoch) + ".pkl"
    torch.save(net, name) 

  0%|          | 0/10 [00:00<?, ?it/s]

0 : 0.0143


 10%|█         | 1/10 [06:26<57:58, 386.47s/it]

0.0018837744242330766
1 : 0.0087


 20%|██        | 2/10 [12:52<51:31, 386.39s/it]

...

0.00031789050167565234
8 : 0.0032


 90%|█████████ | 9/10 [57:58<06:26, 386.52s/it]

0.0002807572682089381
9 : 0.0030


100%|██████████| 10/10 [1:04:24<00:00, 386.48s/it]

0.00023958180247004034
